In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [2]:
import tensorflow_datasets as tfds
(training_set, validation_set),info1=tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'], as_supervised=True, shuffle_files=True,with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
print(info1)
train = training_set.map(
    lambda image, label: (tf.image.resize(image,[224,224]), label)).map(
    lambda image, label: (tf.image.convert_image_dtype(image, tf.float32), tf.reshape(label,[1]))
).cache().map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.0, upper=1.0), label)
).shuffle(
    100
).prefetch(10).batch(
    24
).repeat()
validation=validation_set.map(
    lambda image, label: (tf.image.resize(image,[224,224]), label)).map(
    lambda image, label: (tf.image.convert_image_dtype(image, tf.float32), tf.reshape(label,[1]))
).cache().map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.0, upper=1.0), label)
).shuffle(
    100
).prefetch(10).batch(
    10
).repeat()
for x in train:
  print(x[1].shape)
  break

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='/root/tensorflow_datasets/tf_flowers/3.0.1.incompleteZDP8RG',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)
(24, 1)


In [4]:
vgg = VGG16(input_shape=[224,224] + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(5, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [5]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train,epochs=20, batch_size=24,steps_per_epoch=122,validation_data=validation,validation_steps=40)


Epoch 1/20
122/122 [==============================] - 31s 152ms/step - loss: 6.2892 - accuracy: 0.6759 - val_loss: 5.2502 - val_accuracy: 0.7325
Epoch 2/20
122/122 [==============================] - 17s 124ms/step - loss: 3.5713 - accuracy: 0.7967 - val_loss: 7.0519 - val_accuracy: 0.7625
Epoch 3/20
122/122 [==============================] - 15s 125ms/step - loss: 2.7092 - accuracy: 0.8595 - val_loss: 4.2228 - val_accuracy: 0.8175
Epoch 4/20
122/122 [==============================] - 16s 128ms/step - loss: 2.2365 - accuracy: 0.8788 - val_loss: 6.0524 - val_accuracy: 0.7900
Epoch 5/20
122/122 [==============================] - 16s 132ms/step - loss: 1.5121 - accuracy: 0.8929 - val_loss: 5.7065 - val_accuracy: 0.7875
Epoch 6/20
122/122 [==============================] - 16s 132ms/step - loss: 1.4493 - accuracy: 0.9045 - val_loss: 7.1523 - val_accuracy: 0.7600
Epoch 7/20
122/122 [==============================] - 16s 135ms/step - loss: 1.2896 - accuracy: 0.9087 - val_loss: 7.1651 - val_ac